# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [3]:
# Your code here
import sqlite3
import pandas as pd
conn = sqlite3.connect('data.sqlite')

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [17]:
pd.read_sql("""
SELECT * 
FROM offices
;
""", conn)

,officeCode,city,phone,addressLine1,addressLine2,state,country,postalCode,territory
0,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
1,2,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA
2,3,NYC,+1 212 555 3000,523 East 53rd Street,apt. 5A,NY,USA,10022,NA
3,4,Paris,+33 14 723 4404,43 Rue Jouffroy D'abbans,,,France,75017,EMEA
4,5,Tokyo,+81 33 224 5000,4-1 Kioicho,,Chiyoda-Ku,Japan,102-8578,Japan
5,6,Sydney,+61 2 9264 2451,5-11 Wentworth Avenue,Floor #2,,Australia,NSW 2010,APAC
6,7,London,+44 20 7877 2041,25 Old Broad Street,Level 7,,UK,EC2N 1HN,EMEA
7,27,Boston,+1 977 299 8345,105 Cambridge Street,,MA,USA,02331,NA


In [15]:
pd.read_sql("""
SELECT e.firstName, e.lastName
FROM employees e
JOIN offices o
USING(officeCode)
WHERE o.city = 'Boston';
""", conn)

,firstName,lastName
0,Julie,Firrelli
1,Steve,Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [21]:
# Your code here
pd.read_sql("""
SELECT e.firstName, e.lastName
FROM employees e
JOIN offices o
USING(officeCode)
WHERE employeeNumber IS NULL
GROUP BY city;
""", conn)

,firstName,lastName


## Write 3 questions of your own and answer them

In [27]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""
pd.read_sql("""
SELECT o.city, COUNT(c.customerNumber) AS customer_count
FROM customers c
JOIN employees e
    ON c.salesRepEmployeeNumber = e.employeeNumber
JOIN offices o
    USING(officeCode)
GROUP BY o.city;
""", conn)

,city,customer_count
0,Boston,12
1,London,17
2,NYC,15
3,Paris,29
4,San Francisco,12
5,Sydney,10
6,Tokyo,5


In [50]:
#Question 1

#Get the total number of orders placed by each customer

# Your code here
pd.read_sql("""
SELECT c.customerName, COUNT(o.orderNumber) AS total_orders
    FROM customers c
    LEFT JOIN orders o
        USING(customerNumber)
    GROUP BY c.customerNumber
    ORDER BY total_orders DESC;
""",conn).head()

,customerName,total_orders
0,Euro+ Shopping Channel,26
1,Mini Gifts Distributors Ltd.,17
2,Reims Collectables,5
3,"Down Under Souveniers, Inc",5
4,"Dragon Souveniers, Ltd.",5


In [52]:
#Question 2

#Find the top 5 customers by total spending

# Your code here

pd.read_sql("""
SELECT c.customerName, COUNT(od.quantityOrdered * od.priceEach) AS total_spent
    FROM customers c
    JOIN orders o
        USING(customerNumber)
    JOIN orderdetails od
        USING(orderNumber)
    GROUP BY c.customerNumber
    ORDER BY total_spent DESC;
""",conn).head()

,customerName,total_spent
0,Euro+ Shopping Channel,259
1,Mini Gifts Distributors Ltd.,180
2,"Australian Collectors, Co.",55
3,La Rochelle Gifts,53
4,"AV Stores, Co.",51


In [60]:
#Question 3

# Calculate the total revenue generated by each product line

# Your code here

pd.read_sql("""
SELECT pl.productLine, SUM(od.quantityOrdered * od.priceEach) AS total_revenue
    FROM productlines pl
    JOIN products p
        USING(productLine)
    JOIN orderdetails od
        USING(productCode)
    GROUP BY pl.productLine
    ORDER BY total_revenue DESC;
""",conn)

,productLine,total_revenue
0,Classic Cars,3853922.49
1,Vintage Cars,1797559.63
2,Motorcycles,1121426.12
3,Trucks and Buses,1024113.57
4,Planes,954637.54
5,Ships,663998.34
6,Trains,188532.92


In [64]:
#Identify the busiest month of the year

pd.read_sql("""
SELECT 
    strftime('%m', orderDate) AS order_month,
    COUNT(orderNumber) AS total_orders
FROM orders
GROUP BY order_month
ORDER BY total_orders DESC
LIMIT 1;
""",conn)

,order_month,total_orders
0,11,63


In [61]:
# Find duplicate customer names

pd.read_sql("""SELECT customerName, COUNT(*) AS name_count
FROM customers
GROUP BY customerName
HAVING COUNT(*) > 1;
""", conn)

,customerName,name_count


In [76]:
#Find the percentage of total revenue contributed by each product line

pd.read_sql("""
SELECT pl.productLine,
    ROUND(SUM(od.quantityOrdered * od.priceEach) / (
            SELECT SUM(od2.quantityOrdered * od2.priceEach)
            FROM orderdetails od2) * 100, 2) AS revenue_percentage
FROM productlines pl
JOIN products p
    USING(productLine)
JOIN orderdetails od
    USING(productCode)
GROUP BY pl.productLine
ORDER BY revenue_percentage DESC;
""", conn)

,productLine,revenue_percentage
0,Classic Cars,40.13
1,Vintage Cars,18.72
2,Motorcycles,11.68
3,Trucks and Buses,10.66
4,Planes,9.94
5,Ships,6.91
6,Trains,1.96


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [88]:
# Your code here

pd.read_sql("""
SELECT 
    e.firstName, e.lastName, p.productName
FROM employees e
JOIN customers c
    ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders o
    USING(customerNumber)
JOIN orderdetails od
    USING(orderNumber)
JOIN products p
    USING(productCode);
""", conn)

,firstName,lastName,productName
0,Leslie,Jennings,1958 Setra Bus
1,Leslie,Jennings,1940 Ford Pickup Truck
2,Leslie,Jennings,1939 Cadillac Limousine
3,Leslie,Jennings,1996 Peterbilt 379 Stake Bed with Outrigger
4,Leslie,Jennings,1968 Ford Mustang
...,...,...,...
2991,Martin,Gerard,1954 Greyhound Scenicruiser
2992,Martin,Gerard,1950's Chicago Surface Lines Streetcar
2993,Martin,Gerard,Diamond T620 Semi-Skirted Tanker
2994,Martin,Gerard,1911 Ford Town Car


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [97]:
# Your code here

pd.read_sql("""
SELECT 
    e.firstName, e.lastName, SUM(od.quantityOrdered) AS total_sold
FROM employees e
JOIN customers c
    ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders o
    USING(customerNumber)
JOIN orderdetails od
    USING(orderNumber)
GROUP BY e.lastName
ORDER BY e.lastName, e.firstName;
""", conn)


,firstName,lastName,total_sold
0,Loui,Bondur,6186
1,Larry,Bott,8205
2,Pamela,Castillo,9290
3,Julie,Firrelli,4227
4,Andy,Fixter,6246
5,Martin,Gerard,4180
6,Gerard,Hernandez,14231
7,Leslie,Jennings,11854
8,Barry,Jones,7486
9,Peter,Marsh,6632


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [106]:
# Your code here

pd.read_sql("""
SELECT 
    e.firstName, e.lastName, COUNT(DISTINCT od.productCode) AS distinct_products_sold
FROM employees e
JOIN customers c
    ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders o
    USING(customerNumber)
JOIN orderdetails od
    USING(orderNumber)
GROUP BY e.lastName
HAVING distinct_products_sold >200;
""", conn)

,firstName,lastName,distinct_products_sold


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!